## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

In [673]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME']) # Remove columns "EIN" and "NAME" from the DataFrame


In [ ]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique() # Count unique values in each column 

# Display the count of unique values 
unique_values 

In [ ]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_count = application_df['APPLICATION_TYPE'].value_counts() # Counts the occurrences of each application type 

# Display the value counts
application_type_count

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_type_count[application_type_count < 500].index.tolist() # Replace types wih fewer than 500 occurrences

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts() # Count occurrences of each classification

# Display the vlaue counts
classification_counts

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_g1 = classification_counts[classification_counts > 1] # Filter classifications with counts greater than 1 

# Display value counts
classification_counts_g1

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_to_replace = classification_counts[classification_counts < 100].index.tolist() # Replace classifications with fewer than 100 occurrences

# Replace in dataframe
for cls in classification_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, drop_first=True) # Perform one-hot encoding and drop the first category 

# Display the first few rows of the updated DataFrame
application_df.head()


In [681]:
# Split our preprocessed data into our features and target arrays
x = application_df.drop(columns=["IS_SUCCESSFUL"]) # Features all except the target 
y = application_df["IS_SUCCESSFUL"] # Target is the IS_SUCCESSFUL column 

# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42) # 80% training, 20% testing


In [682]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = x_train.shape[1] # Number of input features (columns in the training dataset)

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=input_features)) # 80 nerons, ReLU activation, input layer specified 

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu')) # 30 nerons, ReLU activation 

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # Single neron, Sigmoid activation for binary classification  

# Check the structure of the model
nn.summary()

In [684]:
# Experiment 1: Increase neurons in the hidden layers
# Updated architecture for better learning capacity
nn = tf.keras.models.Sequential()

# First hidden layer with more neurons
nn.add(tf.keras.layers.Dense(units=100, activation='relu', input_dim=input_features))

# Second hidden layer with more neurons
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


In [685]:
# Experiment 2: Add a third hidden layer
# Introduced additional complexity to capture deeper patterns
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation='relu', input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=25, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


In [686]:
# Experiment 5: Add Dropout to reduce overfitting
from tensorflow.keras.layers import Dropout

nn = tf.keras.models.Sequential()

# First hidden layer with Dropout
nn.add(tf.keras.layers.Dense(units=100, activation='relu', input_dim=input_features))
nn.add(Dropout(0.2))  # Drop 20% of neurons

# Second hidden layer with Dropout
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))
nn.add(Dropout(0.2))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


In [687]:
# Compile the model
nn.compile(optimizer='adam', # Optimizer to adjust weights based on loss function
           loss='binary_crossentropy', # Loss function for binary classification
           metrics=['accuracy']) # Metric to add during training 

In [ ]:
# Train the model
history = nn.fit(x_train_scaled, y_train, # Training data
                 epochs=50, # Number of times the model sees the full dataset
                 batch_size=32, # Numbers of samples per update
                 validation_data=(x_test_scaled, y_test)) # Validation data to monitor accuracy during training 

In [ ]:
# Experiment 3: Train with more epochs
# Increased epochs from 50 to 100 to improve learning over time
history = nn.fit(x_train_scaled, y_train, 
                 epochs=100, 
                 batch_size=32, 
                 validation_data=(x_test_scaled, y_test))


In [ ]:
# Experiment 4: Train with smaller batch size
# Reduced batch size to 16 for finer weight updates
history = nn.fit(x_train_scaled, y_train, 
                 epochs=50, 
                 batch_size=16, 
                 validation_data=(x_test_scaled, y_test))


In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5") # Save the trained model to a file name 'AlphabetSoupCharity.h5'

## Summary

Despite multiple attempts to optimize the neural network, the highest accuracy achieved was **73.2%**, which fell short of the target of **75%**. Here is a brief overview of what was tried and its outcomes:

### Increased Neurons in Hidden Layers
- Added more neurons (100 in the first layer, 50 in the second).
- **Result**: Improved accuracy slightly to **73.0%**.

### Added a Third Hidden Layer
- Introduced a third hidden layer with 25 neurons to increase model complexity.
- **Result**: Accuracy dropped to **72.6%**, possibly due to overfitting.

### Trained with More Epochs
- Increased the number of training epochs from 50 to 100.
- **Result**: Accuracy decreased slightly, likely due to overfitting the training data.

### Reduced Batch Size
- Reduced batch size to 16 for finer weight updates during training.
- **Result**: Accuracy improved slightly to **73.1%**.

### Added Dropout
- Added dropout layers to the model to reduce overfitting by randomly dropping 20% of neurons during training.
- **Result**: Accuracy dropped significantly to **57%**.

## Conclusion

Even after trying different ways to improve the model, the highest accuracy I could get was **73.2%**, which is below the **75%** target. I made several changes, including adding more neurons, using more training epochs, changing the batch size, and grouping rare categories. Some of these changes helped a little, but none were enough to reach the goal.

This shows how tricky it can be to improve a model when working with a dataset like this. Even though the target wasn’t reached, the process helped me understand how adjustments affect the model’s performance.
